#Pràctica 3: Movielens filtratge col·laboratiu

David Galera

Es demana fer un programa (notebook) que:

#a) Implementi el mètode de filtratge col·laboratiu ítem-ítem. Heu de definir una funció que, donada una pel·lícula i un nombre k ens busqui les k pel·lícules més semblants i una altra que, donat l’identificador d’un usuari i un nombre k, ens produeixi un màxim de k recomanacions per l’usuari.

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
import pandas as pd
import numpy as np
from scipy.stats import pearsonr

In [3]:
ratings = pd.read_csv("/content/drive/MyDrive/Advanced ML/ratings.csv")
print('ratings shape:',ratings.shape)
print(ratings.columns)
ratings.head()



ratings shape: (100836, 4)
Index(['userId', 'movieId', 'rating', 'timestamp'], dtype='object')


,userId,movieId,rating,timestamp
0,1,1,4.0,964982703
1,1,3,4.0,964981247
2,1,6,4.0,964982224
3,1,47,5.0,964983815
4,1,50,5.0,964982931


In [4]:
movies = pd.read_csv("/content/drive/MyDrive/Advanced ML/movies.csv")
print('movies shape:',movies.shape)
print(movies.columns)
movies.head()

movies shape: (9742, 3)
Index(['movieId', 'title', 'genres'], dtype='object')


,movieId,title,genres
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy
1,2,Jumanji (1995),Adventure|Children|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance
3,4,Waiting to Exhale (1995),Comedy|Drama|Romance
4,5,Father of the Bride Part II (1995),Comedy


Inserim una columna adicional al DataFrame *ratings* amb el títol de la pel·lícula

In [5]:
ratings = pd.merge(ratings,movies[['movieId','title']],on=['movieId'],how='left')
#print(ratings.dtypes)
ratings.head()

,userId,movieId,rating,timestamp,title
0,1,1,4.0,964982703,Toy Story (1995)
1,1,3,4.0,964981247,Grumpier Old Men (1995)
2,1,6,4.0,964982224,Heat (1995)
3,1,47,5.0,964983815,Seven (a.k.a. Se7en) (1995)
4,1,50,5.0,964982931,"Usual Suspects, The (1995)"


In [6]:
print("Número d'usuaris:",len(ratings.userId.unique()))#610 usuaris
print("Número de pel·lícules:",len(ratings.title.unique()))#9719 pel·lícules

Número d'usuaris: 610
Número de pel·lícules: 9719


Creem un DataFrame on cada instància representa una pel·lícula, cada columna representa un usuari i els valors corresponen als ratings dels l'usuaris a les pel·lícules.

In [105]:
df=ratings.pivot_table(values='rating',index='title',columns='userId')#,fill_value=0)
df.head()

userId,1,2,3,4,5,6,7,8,9,10,...,601,602,603,604,605,606,607,608,609,610
title,,,,,,,,,,,,,,,,,,,,,
'71 (2014),NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,4.0
'Hellboy': The Seeds of Creation (2004),NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
'Round Midnight (1986),NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
'Salem's Lot (2004),NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
'Til There Was You (1997),NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


Calcularem la similitud entre una pel·lícula i una altra mitjançant els ratings dels usuaris que han puntuat les dues pel·lícules i mesurant la correlació que hi ha en les puntuacions que han donat mitjançant la **correlació de pearson**, posteriorment imprimirem les pel·licules semblants ordenades segons aquesta correlació, per fer-ho definim una funció on cal entrar el títol d'una pel·lícula **P** i el nombre **k** de recomanacions, la funció realitza el següent:
- Itera entre la resta de pel·lícules.
- Per a cada pel·lícula **P'** itera entre tots els usuaris que han puntuat la pel·lícula **P** i comprova que l'usuari **i** ha puntuat també **P'**. Si es compleix la condició, enmagatzema el *rating* de l'usuari **i** a **P** en una llista **r_u** i a **P'** en una llista **r_v**.
- Computa la **correlació de pearson** entre **r_u** i **r_v** si les pel·lícules han estat puntuades per més de 2 usuaris comuns.
- Enmagatzema en un diccionari en format {clau:valor} la pel·lícula **P'** i la **correlació de pearson**.
- Imprimeixi per pantalla les **k** pel·lícules **P'** amb **correlació de pearson** més alta.

In [112]:
def recomanacions():
  print('Escriu un títol: ')
  pelicula=str(input())
  print('Número de títols recomanats: ')
  k=int(input())
  from operator import itemgetter
  similar_films=df.drop([pelicula],axis=0)

  d={}
  mean_u=df.loc[pelicula][df.loc[pelicula].notnull()].mean()#mitjana de ratings de la pel·lícula introduida
################################################################################
#Càlcul de similituds entre la pel·lícula introduïda i la resta
  for film in similar_films.index:
    mean_v=df.loc[film][df.loc[film].notnull()].mean()#mitjana de ratings de la pel·lícula similar
    r_u=[]#llista amb el rating de l'usuari "i" a la pel·lícula introduïda
    r_v=[]#llista amb el rating de l'usuari "i" a la pel·lícula similar
    for i in df.loc[pelicula][df.loc[pelicula].notnull()].index:#iterem entre tots els usuaris que han puntuat la pel·lícula introduïda
      if ~np.isnan(df.loc[film][i]):#si l'usuari "i" també ha puntuat la pel·lícula similar
        r_u.append(df.loc[pelicula][i])#afegim el rating de l'usuari "i" a la llista introduïda
        r_v.append(df.loc[film][i])#afegim el rating de l'usuari "i" a la llista similar
    if len(r_u)>1:#Almenys 2 usuaris han d'haver puntuat les dues pel·lícules per calcular-ne la correlació
      num=0
      den_1=0
      den_2=0
      for index in range(len(r_u)):
        num+=(r_u[index]-mean_u)*(r_v[index]-mean_v)#numerador del coeficient de correlació de pearson
        den_1+=(r_u[index]-mean_u)**2
        den_2+=(r_v[index]-mean_v)**2
      cor=num/((den_1**0.5)*(den_2**0.5))#càlcul del coeficient de correlació de pearson
      d[film]=cor
################################################################################
#IMPRESSIÓ DE LES K PEL·LÍCULES AMB LA CORRELACIÓ MÉS ALTA
  if k>len(d):#màxim titols recomanats és el total de pel·lícules predites
    k=len(d)
  print("Llistat de títols recomanats")
  print('--------------------------------------------------------')
  for recomanacio in dict(sorted(d.items(), key = itemgetter(1), reverse = True)[:k]):
    print(recomanacio,', Coeficient de pearson: ',dict(sorted(d.items(), key = itemgetter(1), reverse = True)[:k])[recomanacio].round(3))
    print('--------------------------------------------------------')

Invoquem la funció amb la pel·lícula *Jumanji (1995)* i *k=15*

In [113]:
recomanacions()
#Jumanji (1995)

Escriu un títol: 
Jumanji (1995)
Número de títols recomanats: 
15
Llistat de títols recomanats
--------------------------------------------------------
Out of Sight (1998) , Coeficient de pearson:  0.935
--------------------------------------------------------
Jungle2Jungle (a.k.a. Jungle 2 Jungle) (1997) , Coeficient de pearson:  0.935
--------------------------------------------------------
Hedwig and the Angry Inch (2000) , Coeficient de pearson:  0.928
--------------------------------------------------------
Change-Up, The (2011) , Coeficient de pearson:  0.921
--------------------------------------------------------
Angels & Demons (2009) , Coeficient de pearson:  0.909
--------------------------------------------------------
About Time (2013) , Coeficient de pearson:  0.908
--------------------------------------------------------
Blood Simple (1984) , Coeficient de pearson:  0.901
--------------------------------------------------------
The Nice Guys (2016) , Coeficient de pearso

Per fer el segon apartat, implementarem una funció a la que cal passar el número **Id** d'usuari i el número **k** de recomanacions desitjades, la funció realitza el següent:
- Crea 2 llistes, amb les pel·lícules vistes i no vistes per l'usuari.
- Per cada vector amb ratings de la pel·lícula no vista, itera entre tots els vectors de pel·lícules vistes i en calcula la **correlació de pearson** a partir del **vector de ratings dels usuaris que han vist les dues pel·lícules** i el rating promig de cada pel·lícula.
- Enmagatzema la correlació entre la pel·lícula no vista i la vista en un diccionari amb el format {Pel·lícula vista:correlació}.
- Amb un **veinatge V=20**, prediu el rating que donaria l'usuari introduït a la **pel·lícula no vista** calculant el rating promig que ha donat a les **20 pel·lícules vistes amb més correlació**, ponderat per la seva simil·litud amb la pel·lícula no vista. Enmagatzema la predicció del rating de la pel·lícula no vista en un diccionari amb el format {Pel·lícula no vista:predicció}.
- Repeteix el procés per totes les pel·lícules no vistes.
- Imprimeix les **k** pel·lícules no vistes amb predicció més alta.

L'operació de computar les pel·lícules vistes més similars per a cada pel·lícula no vista té un cost computacional molt alt, degut a que hi ha casi 10000 pel·lícules i molts usuaris en tenen moltes de no vistes. Com que el temps de comput és molt alt, barregem el DataFrame i utilitzem 4000 pel·lícules.

In [75]:
df_suffle=df.sample(frac = 1)

In [82]:
df = df_suffle.iloc[:4000,:]
df.head()

userId,1,2,3,4,5,6,7,8,9,10,...,601,602,603,604,605,606,607,608,609,610
title,,,,,,,,,,,,,,,,,,,,,
Stepmom (1998),NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,3.5,NaN,NaN,NaN,NaN
"Twilight Saga: New Moon, The (2009)",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3.0,...,NaN,NaN,NaN,NaN,3.0,NaN,NaN,NaN,NaN,NaN
Vice (2015),NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
"O Brother, Where Art Thou? (2000)",NaN,NaN,NaN,3.0,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,3.0,NaN,NaN,4.5,NaN,4.5,NaN,4.5
"Turning Point, The (1977)",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,4.0,NaN,NaN,NaN,NaN


In [91]:
def itemitem():
  from operator import itemgetter
  V=20#veinatge
  print('Escriu userId: ')
  user=int(input())
  print('Número de títols recomanats: ')
  k=int(input())
  
  non_rated_films=df.drop(df[user][df[user].notnull()].index,axis=0).index#Llista amb les pel·lícules no vistes per l'usuari
  rated_films=df[user][df[user].notnull()].index#Llista amb les pel·lícules vistes per l'usuari
  prediction={}

#  counter = 0
  for novista in non_rated_films:#iterem entre les pel·lícules NO VISTA
#    counter += 1
#    if counter == 500:#Fer-ho per 4000 pel·lícules
#        break

    mean_u=df.loc[novista][df.loc[novista].notnull()].mean()#mitjana de ratings de la pel·lícula NO VISTA
################################################################################
#Càlcul de similituds entre la pel·lícula NO VISTA i totes les VISTES
    d={}
    for vista in rated_films:#iterem entre les pel·lícules VISTES
      mean_v=df.loc[vista][df.loc[vista].notnull()].mean()#mitjana de ratings de la pel·lícula VISTA
      r_u=[]#llista amb el rating de l'usuari "i" a la pel·lícula NO VISTA
      r_v=[]#llista amb el rating de l'usuari "i" a la pel·lícula VISTA
      for i in df.loc[novista][df.loc[novista].notnull()].index:#iterem entre tots els usuaris que han puntuat la pel·lícula NOVISTA
        if ~np.isnan(df.loc[vista][i]):#si l'usuari "i" també ha puntuat la pel·lícula VISTA:
          r_u.append(df.loc[novista][i])#afegim el rating de l'usuari "i" a la llista NO VISTA
          r_v.append(df.loc[vista][i])#afegim el rating de l'usuari "i" a la llista VISTA
      if len(r_u)>1:#almenys 2 usuaris han d'haver votat les dues pel·lícules per poder mesurar la correlació
        num=0
        den_1=0
        den_2=0
        for index in range(len(r_u)):
          num+=(r_u[index]-mean_u)*(r_v[index]-mean_v)#numerador del coeficient de correlació de pearson
          den_1+=(r_u[index]-mean_u)**2
          den_2+=(r_v[index]-mean_v)**2
        cor=num/((den_1**0.5)*(den_2**0.5))#càlcul del coeficient de correlació de pearson
#guardem en un diccionari la similitud entre la pel·lícula VISTA i la NO VISTA
# per l'usuari en format {pel·licula vista:coef pearson}        
        if (~np.isnan(cor))&(cor>0):
          d[vista]=cor
################################################################################
#CALCUL DE LA PREDICCIÓ PER LES PEL·LÍCULES NO VISTES

    if len(d)>=V:#Si l'usuari ha vist un mínim de V pel·lícules similars, procedim a fer la predicció
      denominador=0
      numerador=0
  #càlcul prediccio del rating que donaria l'usuari a la pel·lícula NO VISTA
  #i l'enmagatzema en un diccionari en format {NO VISTA: predicció} amb un veinatge de 20
      for vei in dict(sorted(d.items(), key = itemgetter(1), reverse = True)[:V]):
  #similitud pel·licula VISTA*RATING USUARI
        numerador+=dict(sorted(d.items(), key = itemgetter(1), reverse = True)[:V])[vei]*df.loc[vei][user]
  #Ponderació similituds      
        denominador+=dict(sorted(d.items(), key = itemgetter(1), reverse = True)[:V])[vei]
  #Predicció del rating de la pel·lícula NO VISTA per l'usuari
      prediction[novista]=numerador/denominador 
################################################################################
#IMPRESSIÓ DE LES K PEL·LÍCULES AMB LA PREDICCIÓ MÉS ALTA

  print("Llistat de títols recomanats")
  print('--------------------------------------------------------')
  if k>len(prediction):#maxim titols recomanats és el total de pel·lícules predites
    k=len(prediction)
  for recomanacio in dict(sorted(prediction.items(), key = itemgetter(1), reverse = True)[:k]):
      print(recomanacio,', Predicció rating: ',dict(sorted(prediction.items(), key = itemgetter(1), reverse = True))[recomanacio].round(3))
      print('--------------------------------------------------------')

L'executarem amb l'usuari 1 que ha vist les següents pel·lícules:

In [104]:
df[1][df[1].notnull()].nlargest(60)


title
Rushmore (1998)                                          5.0
Dumb & Dumber (Dumb and Dumber) (1994)                   5.0
American Tail, An (1986)                                 5.0
Goodfellas (1990)                                        5.0
Rescuers, The (1977)                                     5.0
Bottle Rocket (1996)                                     5.0
Seven (a.k.a. Se7en) (1995)                              5.0
Thin Red Line, The (1998)                                5.0
Grumpy Old Men (1993)                                    5.0
Tommy Boy (1995)                                         5.0
American Beauty (1999)                                   5.0
Newton Boys, The (1998)                                  5.0
All Quiet on the Western Front (1930)                    5.0
Full Metal Jacket (1987)                                 5.0
Young Frankenstein (1974)                                5.0
Indiana Jones and the Last Crusade (1989)                5.0
Austin Powers: Int

In [92]:
itemitem()

Escriu userId: 
1
Número de títols recomanats: 
50


<ipython-input-91-310a12e387c8>:39: RuntimeWarning: invalid value encountered in double_scalars
  cor=num/((den_1**0.5)*(den_2**0.5))#càlcul del coeficient de correlació de pearson


Llistat de títols recomanats
--------------------------------------------------------
Saw IV (2007) , Predicció rating:  4.87
--------------------------------------------------------
Salo, or The 120 Days of Sodom (Salò o le 120 giornate di Sodoma) (1976) , Predicció rating:  4.839
--------------------------------------------------------
Marty (1955) , Predicció rating:  4.829
--------------------------------------------------------
Three Colors: Blue (Trois couleurs: Bleu) (1993) , Predicció rating:  4.814
--------------------------------------------------------
Heist (2001) , Predicció rating:  4.813
--------------------------------------------------------
Highlander III: The Sorcerer (a.k.a. Highlander: The Final Dimension) (1994) , Predicció rating:  4.8
--------------------------------------------------------
Twilight Samurai, The (Tasogare Seibei) (2002) , Predicció rating:  4.798
--------------------------------------------------------
Lupin III: The Castle Of Cagliostro (Rupan 

#b/ Implementi el mètode de filtratge col·laboratiu user-user. Heu de definir una funció que, donat l’identificador d’un usuari i un nombre k, ens produeixi un màxim de k recomanacions per l’usuari. Compareu qualitativement els resultats amb els obtinguts a l’apartat a)

Per realitzar l'apartat treballem amb el DataFrame amb format files=usuaris i columnes=pel·lícules.

La funció consta d'etapes ben diferenciades:

- Primer s'introdueix la **id de l'usuari** i el número **k de recomanacions** que es volen, la funció defineix una llista amb les pel·lícules que no ha vist l'usuari, i una altra llista amb les que si ha vist. També defineix una llista amb tots els usuaris excepte l'usuari introduït.

- La segona part de la funció calcula la correlació de pearson entre el vector de l'usuari introduït i la resta amb un procés iteratiu. Enmagatzema en un diccionari amb format {id:correlació} els resultats i crea una llista **user_cor** amb les id dels usuaris ordenada segons correlació.

- La tercera part tracta d'iterar entre totes les pel·lícules no vistes i preedir el rating utilitzant un **veinatge de 30**, això és calcular la mitjana ponderada dels 30 primers elements de la llista **user_cor** que han vist la pel·lícula. Posteriorment s'enmagatzemen les dades en un diccionari amb format {pel·lícula no vista:predicció}.

- Finalment s'imprimeixen les **k** pel·lícules amb la predicció més alta.



In [76]:
df2 = df_suffle.iloc[:4000,:].T
df2.head()

title,Stepmom (1998),"Twilight Saga: New Moon, The (2009)",Vice (2015),"O Brother, Where Art Thou? (2000)","Turning Point, The (1977)",House Party (1990),Eragon (2006),Poker Night (2014),Bad Karma (2012),"Scotland, Pa. (2001)",...,Love Actually (2003),Pi (1998),Maximum Ride (2016),THX 1138 (1971),"Safety of Objects, The (2001)",Righteous Kill (2008),Savages (2012),Vertical Limit (2000),Cat People (1982),Garage Days (2002)
userId,,,,,,,,,,,,,,,,,,,,,
1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,NaN,NaN,NaN,3.0,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [79]:
def user_user():
  from operator import itemgetter
  print('Escriu userId: ')
  user=int(input())
  print('Número de títols recomanats: ')
  k=int(input())

  rated_films=list(df2.loc[user][df2.loc[user].notnull()].index)#Llista amb les pel·lícules vistes per l'usuari
  non_rated_films=list(set(df2.columns)-set(rated_films))#Llista amb les pel·lícules no vistes per l'usuari

  #Creem una llista amb tots els usuaris excepte l'introduit
  rest_users = [x for x in list(df2.index) if x!=user]
  #print(rest_users)
  mean_u=df2.loc[user][df2.loc[user].notnull()].mean()
  #print(mean_u)
  ################################################################################
  #CÀLCUL DE SIMILITUD ENTRE USUARIS:

  d={}
  for id in rest_users:
    r_u=[] 
    r_v=[]
    mean_v=df2.loc[id][df2.loc[id].notnull()].mean()
    for film in rated_films:
      if ~np.isnan(df2.loc[id][film]):#si els 2 usuaris han puntuat la pel·lícula
        r_u.append(df2.loc[user][film])#afegim el rating de l'usuari especificat a la llista
        r_v.append(df2.loc[id][film])#afegim el rating de l'usuari "id" a la llista
  #els 2 usuaris han d'haver puntuat almenys 2 pel·lícules en comú per poder calcular la correlació entre ells
    if len(r_u)>1:
      num=0
      den_1=0
      den_2=0
      for index in range(len(r_u)):
        num+=(r_u[index]-mean_u)*(r_v[index]-mean_v)#numerador del coeficient de correlació de pearson
        den_1+=(r_u[index]-mean_u)**2
        den_2+=(r_v[index]-mean_v)**2
      cor=num/((den_1**0.5)*(den_2**0.5))#càlcul del coeficient de correlació de pearson
      if (~np.isnan(cor))&(cor>0):
        d[id]=cor.round(6)#guardem en un diccionari la similitud entre usuaris

  #llistat ORDENAT amb els usuaris més similars
  user_cor=[x[0] for x in sorted(d.items(), key = itemgetter(1), reverse = True)]
  ################################################################################
  #CALCUL DE LA PREDICCIÓ PER LES PEL·LÍCULES NO VISTES

  prediction={}
  for novista in non_rated_films:#iterem les pel·lícules no vistes
    V=30#veinatge
  #emmagatzemem els veïns que han vist la pel·lícula per ordre de SIMILITUD
    veins = [x for x in user_cor if ~np.isnan(df2.loc[x][novista])]
  #Càlcul de la predicció
    if len(veins)>=V:#Si hi ha algun veí a la llista
  #    V=min(V,len(veins))
  #    print(veins)
      denominador=0
      numerador=0
      for neighbor in veins[:V]:
  #      print(neighbor)
  #      print(novista)
        numerador+=d[neighbor]*df2.loc[neighbor][novista]#sumatori de similituds*Rating
        denominador+=d[neighbor]#sumatori de similituds
  #    print(numerador)
  #    print(denominador)
      prediction[novista]=numerador/denominador #Predicció del rating per la pel·lícula NO VISTA per l'usuari
  ################################################################################
  #IMPRESSIÓ DE LES K PEL·LÍCULES AMB LA PREDICCIÓ MÉS ALTA
  print("Llistat de títols recomanats")
  print('--------------------------------------------------------')
  if k>len(prediction):#màxim titols recomanats és el total de pel·lícules predites
    k=len(prediction)
  for recomanacio in dict(sorted(prediction.items(), key = itemgetter(1), reverse = True)[:k]):
      print(recomanacio,', Predicció rating: ',dict(sorted(prediction.items(), key = itemgetter(1), reverse = True))[recomanacio].round(3))
      print('--------------------------------------------------------')


Invoquem la funció amb l'usuari 1 i k=50:

In [80]:
user_user()

Escriu userId: 
1
Número de títols recomanats: 
50


<ipython-input-79-4f16c04ac298>:39: RuntimeWarning: invalid value encountered in double_scalars
  cor=num/((den_1**0.5)*(den_2**0.5))#càlcul del coeficient de correlació de pearson


Llistat de títols recomanats
--------------------------------------------------------
Shawshank Redemption, The (1994) , Predicció rating:  4.562
--------------------------------------------------------
Lord of the Rings: The Two Towers, The (2002) , Predicció rating:  4.367
--------------------------------------------------------
Shaun of the Dead (2004) , Predicció rating:  4.327
--------------------------------------------------------
Eternal Sunshine of the Spotless Mind (2004) , Predicció rating:  4.318
--------------------------------------------------------
North by Northwest (1959) , Predicció rating:  4.3
--------------------------------------------------------
To Kill a Mockingbird (1962) , Predicció rating:  4.294
--------------------------------------------------------
Brazil (1985) , Predicció rating:  4.283
--------------------------------------------------------
Dead Poets Society (1989) , Predicció rating:  4.274
--------------------------------------------------------


En un DataFrame amb casi 10000 pel·lícules i només 610 usuaris és molt més ràpid calcular la similitud entre usuaris que entre pel·lícules, perque hi ha molts usuaris amb moltes pel·lícules no vistes.

Les pel·lícules són diferents a les recomanades amb el mètode item-item.